In [1]:
# TODO
# 1. weight on same class different type 
#     (predicting same class but different type should have a smaller error than different class)
# 2. CNN
# 3. Error analysis, by angle, by style, by randomness, weighted f1, confusion matrix
# 4. error plots vs correctly predicted plots
# 5. shap analysis 
# 6. transfer learning by vgg-16, trainable & not trainable
# 7. use CV features as first layer
# 8. PCA on raw pixels, check live session excercise 12
# 9. maybe add KNN to classical models, use PCA & T-SNE hog features, similar to live session excercise 12
# 10. visualize CNN filters
# 11. identify hard samples in val set to do training analysis

# Imports

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Reshape, MaxPooling2D, Flatten, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.constraints import UnitNorm

2023-04-12 17:39:28.884071: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 17:39:29.022348: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 17:39:29.623495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 17:39:29.625536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 17:39:31.302383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np

from nebula.data.yg_ar.setup_data_image_hard import read_data
from nebula.common import to_scale_one, write_pickle, read_pickle
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import os
import os.path as osp
import numpy as np
from sklearn.metrics import classification_report

# Read data and apply label map

In [4]:
def create_label_map(labels):
    label_set = set()
    for lt in labels:
        label_set.add(lt)

    label_map = {}
    count = 0
    for l in label_set:
        label_map[l] = count
        count += 1
        
    return label_map

In [5]:
df_path = "/home/ubuntu/data/yg_ar/image_medium_df.pkl"
random_seed = 1
df, train_df, test_df, valid_df = read_data(df_path, random_seed)

In [6]:
label_a_map = create_label_map(df["label_a"])


In [7]:
X_train = np.expand_dims(np.array([x for x in train_df.image]),axis=3)
y_train = train_df.label_a.map(label_a_map).to_list()

X_val = np.expand_dims(np.array([x for x in valid_df.image]),axis=3)
y_val = valid_df.label_a.map(label_a_map).to_list()

X_test = np.expand_dims(np.array([x for x in test_df.image]),axis=3)
y_test = test_df.label_a.map(label_a_map).to_list()

# Scale data

In [8]:
scaled_X_train = X_train/255.0 
scaled_X_val = X_val/255.0
scaled_X_test = X_test/255.0

# Define CNN

In [9]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm(), \
         input_shape = scaled_X_train[0].shape))
model.add(Conv2D(32, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(Conv2D(64, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(128, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(256, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(512, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))
model.compile(Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=3,padding='same', input_shape = scaled_X_train[0].shape))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Conv2D(64, kernel_size=3,padding='same'))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Conv2D(128, kernel_size=3,padding='same'))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(40,activation='softmax'))
# model.compile(Adam(learning_rate=0.0001,clipnorm=1.0),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 156, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 156, 156, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 78, 78, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 78, 78, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 78, 78, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 78, 78, 64)        3

# Train model

In [12]:
es = EarlyStopping(monitor='val_accuracy',min_delta=0.001, patience=5, restore_best_weights=True)

In [13]:
model.fit(scaled_X_train, to_categorical(y_train),\
          batch_size=4,\
          epochs=100,\
          validation_data = (scaled_X_val,to_categorical(y_val)),\
          callbacks=[es])

Epoch 1/100
4020/4020 [==============================] - 495s 123ms/step - loss: 1.0022 - accuracy: 0.6552 - val_loss: 0.3082 - val_accuracy: 0.8881
Epoch 2/100
4020/4020 [==============================] - 493s 123ms/step - loss: 0.1657 - accuracy: 0.9484 - val_loss: 0.2922 - val_accuracy: 0.9222
Epoch 3/100
4020/4020 [==============================] - 492s 122ms/step - loss: 0.0915 - accuracy: 0.9711 - val_loss: 0.1541 - val_accuracy: 0.9472
Epoch 4/100
4020/4020 [==============================] - 491s 122ms/step - loss: 0.0595 - accuracy: 0.9810 - val_loss: 0.0924 - val_accuracy: 0.9733
Epoch 5/100
4020/4020 [==============================] - 494s 123ms/step - loss: 0.0570 - accuracy: 0.9840 - val_loss: 0.1291 - val_accuracy: 0.9648
Epoch 6/100
4020/4020 [==============================] - 495s 123ms/step - loss: 0.0414 - accuracy: 0.9876 - val_loss: 0.0882 - val_accuracy: 0.9710
Epoch 7/100
4020/4020 [==============================] - 547s 136ms/step - loss: 0.0336 - accuracy: 0.9904

# Test model

In [14]:
y_pred = np.argmax(model.predict(scaled_X_test),axis=1)
print(classification_report(y_test,y_pred))

68/68 [==============================] - 16s 236ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       216
           1       1.00      0.99      0.99       216
           2       1.00      0.99      0.99       216
           3       1.00      1.00      1.00       216
           4       1.00      1.00      1.00       216
           5       1.00      1.00      1.00       216
           6       0.99      1.00      0.99       216
           7       1.00      1.00      1.00       216
           8       0.99      1.00      0.99       216
           9       1.00      0.99      1.00       216

    accuracy                           1.00      2160
   macro avg       1.00      1.00      1.00      2160
weighted avg       1.00      1.00      1.00      2160



In [15]:
model.save('./cnn_medium_action_only.h5')